<a href="https://colab.research.google.com/github/TABEYWICKRAMA/BigData/blob/main/Neo4j_Notebook_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## install Neo4j 

In [9]:
!pip install neo4j

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


## Connect to the Database

In [10]:
from neo4j import GraphDatabase

# URI examples: "neo4j://localhost", "neo4j+s://xxx.databases.neo4j.io"
URI = "neo4j+s://42f16413.databases.neo4j.io"
AUTH = ("neo4j", "gUne6S34xEyCTEtLYuyZ64NACH4Gplo-Qvx8Rm7GKhw")
print("connection successfull !!!")

with GraphDatabase.driver(URI, auth=AUTH) as driver:
    driver.verify_connectivity()

connection successfull !!!


## Query the database


*   Write to the database




In [15]:
def create_person(tx, name):  
    result = tx.run(
        "MERGE (:Person {name: $name})",  
        name=name  
    )
    summary = result.consume()
    return summary

with driver.session(database="neo4j") as session:
    summary = session.execute_write(create_person, name="Alice")  

    print("Created {nodes_created} nodes in {time} ms.".format(
        nodes_created=summary.counters.nodes_created,
        time=summary.result_available_after
    ))

Created 0 nodes in 6 ms.


*   Read from the database



In [16]:
def get_people(tx):
    result = tx.run("MATCH (p:Person) RETURN p.name AS name")
    records = list(result)  # a list of Record objects
    summary = result.consume()
    return records, summary

with driver.session(database="neo4j") as session:
    records, summary = session.execute_read(get_people)

    # Summary information
    print("The query `{query}` returned {records_count} records in {time} ms.".format(
        query=summary.query, records_count=len(records),
        time=summary.result_available_after
    ))

    # Loop through results and do something with them
    for person in records:
        print(person.data())  # obtain record as dict

The query `MATCH (p:Person) RETURN p.name AS name` returned 1 records in 3 ms.
{'name': 'Alice'}




*   Update the database



In [17]:
def update_person(tx, name, age):
    result = tx.run("""
        MATCH (p:Person {name: $name})
        SET p.age = $age
        """, name=name, age=age
    )
    summary = result.consume()
    return summary

with driver.session(database="neo4j") as session:
    summary = session.execute_write(update_person, name="Alice", age=42)
    print(f"Query counters: {summary.counters}.")

Query counters: {'properties_set': 1}.




*   Delete from the database



In [19]:
def delete_person(tx, name):
    result = tx.run("""
        MATCH (p:Person {name: $name})
        DETACH DELETE p
        """, name="Alice"
    )
    summary = result.consume()
    return summary

with driver.session(database="neo4j") as session:
    summary = session.execute_write(delete_person, name="Alice")
    print(f"Query counters: {summary.counters}.")

Query counters: {'nodes_deleted': 1}.
